In [1]:
%load_ext cython

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import ast
import re
import time
import inspect

from pymoo.indicators.hv import Hypervolume

def calculate_hypervolume(archive_objs, ref_point=None):
    if ref_point is None:
        ref_point = np.max(archive_objs, axis=0) + 1.0
    hv = Hypervolume(ref_point=ref_point)
    return hv(archive_objs)

In [3]:
def load_profits_from_file(filename, num_objectives, num_items):
    profits = np.zeros((num_objectives, num_items))
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    idx = 1
    for obj in range(num_objectives):
        idx += 1
        for item in range(num_items):
            idx += 1
            idx += 1
            profit = int(lines[idx])
            profits[obj, item] = profit
            idx += 1
    return profits

def load_weights_from_file(filename, num_objectives, num_items):
    weights = np.zeros((num_objectives, num_items), dtype=int)
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    idx = 1
    for obj in range(num_objectives):
        idx += 1
        for item in range(num_items):
            idx += 1
            weight = int(lines[idx])
            weights[obj, item] = weight
            idx += 2
    return weights

def load_capacities_from_file(filename, num_objectives, num_items):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    idx = 1
    capacities = []
    for obj in range(num_objectives):
        cap = float(lines[idx])
        capacities.append(cap)
        idx += 1 + 3 * num_items
    return np.array(capacities)

In [4]:
def is_dominated(point, others):
    return np.any(np.all(others >= point, axis=1) & np.any(others > point, axis=1))

def hypervolume(points, ref_point=None):
    if len(points) == 0:
        return 0.0
    if ref_point is None:
        ref_point = np.max(points, axis=0) + 100
    points = points[points[:,0].argsort()]
    hv = 0.0
    prev_y = ref_point[1]
    for x, y in points:
        hv += abs(prev_y - y) * abs(ref_point[0] - x)
        prev_y = y
    return hv

def pareto_gaps(points, bins=10):
    if points is None or len(points) == 0:
        return []
    xs, ys = points[:,0], points[:,1]
    hist, xedges, yedges = np.histogram2d(xs, ys, bins=bins)
    gaps = np.argwhere(hist == 0)
    return list(gaps)

globals()['is_dominated'] = is_dominated
globals()['hypervolume'] = hypervolume
globals()['pareto_gaps'] = pareto_gaps

In [5]:
def prompt_llm_for_move_func(
    archive_objs, gaps, agent_stats, llm_endpoint, previous_code=None, feedback=None
):
    improved_example = """
def llm_move(observation, profits, weights, capacities, archive_objs):
    import numpy as np
    observation = np.array(observation, dtype=int)
    profits = np.array(profits)
    weights = np.array(weights)
    capacities = np.array(capacities)
    items_selected = [i for i, x in enumerate(observation) if x == 1]
    items_unselected = [i for i, x in enumerate(observation) if x == 0]
    best_gain = float('-inf')
    best_move = (-1, -1)
    for remove_idx in items_selected:
        for add_idx in items_unselected:
            candidate = observation.copy()
            candidate[remove_idx] = 0
            candidate[add_idx] = 1
            candidate = np.array(candidate, dtype=int)
            total_weights = np.sum(weights * candidate, axis=1)
            if np.all(total_weights <= capacities):
                gain = np.sum(profits[:, add_idx]) - np.sum(profits[:, remove_idx])
                if gain > best_gain:
                    best_gain = gain
                    best_move = (remove_idx, add_idx)
    return best_move
"""
    prompt = (
        "You are a metaheuristic designer for multi-objective knapsack optimization.\n"
        "Write ONLY Python code for the following function, and nothing else:\n\n"
        "def llm_move(observation, profits, weights, capacities, archive_objs):\n"
        "    '''\n"
        "    - Must use archive_objs to check Pareto improvement and hypervolume.\n"
        "    - Only select feasible moves that are non-dominated and increase hypervolume.\n"
        "    - Always end with 'return best_move'.\n"
        "    '''\n"
        "Here is an improved example to follow:\n"
        f"{improved_example}\n"
    )
    if previous_code:
        prompt += "\nPrevious move function code:\n" + previous_code + "\n"
    if feedback:
        prompt += "\nFeedback on previous moves:\n" + feedback + "\n"
    prompt += "\nReturn ONLY the function definition above, with no explanation, markdown, or extra text."

    try:
        response = requests.post(
            llm_endpoint,
            json={
                "model": "llama3",
                "prompt": prompt,
                "stream": False
            }
        )
        text = response.json()["response"]
        print("LLM full text:", text)
        text = text.replace("```python", "").replace("```", "").strip()
        code_match = re.search(
            r"def llm_move\([^\)]*\):(?:\n|.)*?return best_move", text
        )
        if code_match:
            code = code_match.group(0)
            return code
        else:
            print("LLM output did not match expected multi-objective function.")
            print("Raw response:", text)
    except Exception as e:
        print("LLM error:", e)
    return None

In [6]:
import numpy as np
from typing import Tuple, Dict, Any, Optional

class MHREGSESLlmAgent:
    def __init__(
        self,
        num_items,
        num_objectives,
        profits=None,
        weights=None,
        capacities=None,
        meta_interval=1,  # reflect every iteration for demo
        llm_endpoint=None,
        burn_in=0,
        verbose=True
    ):
        self.num_items = num_items
        self.num_objectives = num_objectives
        self.profits = profits
        self.weights = weights
        self.capacities = capacities
        self.meta_interval = meta_interval
        self.llm_endpoint = llm_endpoint or "http://localhost:11434/api/generate"
        self.verbose = verbose
        self.generation = 0
        self.burn_in = burn_in
        self.move_types = ["local_search", "mutation", "diversity", "llm_generated"]
        self.move_funcs = {
            "local_search": self.local_search_move,
            "mutation": self.mutation_move,
            "diversity": self.diversity_move
        }
        self.llm_generated_func = None
        self.llm_code_history = []
        self.move_counts = {k: 1 for k in self.move_types}
        self.move_successes = {k: 1 for k in self.move_types}
        self.move_probs = np.array([0.33, 0.33, 0.33, 0.01], dtype=np.float64)
        self.move_probs = self.move_probs / np.sum(self.move_probs)
        self.best_observation = None
        self.hypervolume_history = []
        self.max_hv_length = 30
        self.last_feedback = None
        self.pareto_improvements = 0

    def observe(self, state):
        return np.array(state["Items"], dtype=np.int32)
    def observe_batch(self, states):
        return [self.observe(state) for state in states]
    def act(self, observation, context=None):
        archive_objs = context.get("archive_objs") if context else None
        move_idx = np.random.choice(len(self.move_types), p=self.move_probs)
        move_type = self.move_types[move_idx]
        move = None
        success = False
        improvement = False
        if move_type == "llm_generated" and self.llm_generated_func is not None:
            try:
                move = self.llm_generated_func(
                    observation, self.profits, self.weights, self.capacities, archive_objs
                )
                if self.verbose:
                    print("LLM-generated move:", move)
            except Exception as e:
                print("LLM move error:", e)
        elif move_type in self.move_funcs:
            move = self.move_funcs[move_type](observation, archive_objs)
        if move is not None and self._is_feasible(observation, move):
            new_obs = observation.copy()
            remove_idx, add_idx = move
            new_obs[remove_idx] = 0
            new_obs[add_idx] = 1
            new_objs = np.sum(self.profits * new_obs, axis=1)
            if archive_objs is not None and len(archive_objs) > 0:
                improvement = not is_dominated(new_objs, archive_objs)
            else:
                improvement = False
            if improvement:
                self.pareto_improvements += 1
            self.report_move_result(move_type, True, improvement)
            return move
        else:
            self.report_move_result(move_type, False, False)
            for fallback_type in self.move_types:
                if fallback_type == move_type:
                    continue
                improvement = False
                if fallback_type == "llm_generated" and self.llm_generated_func is not None:
                    try:
                        move = self.llm_generated_func(
                            observation, self.profits, self.weights, self.capacities, archive_objs
                        )
                    except Exception as e:
                        continue
                elif fallback_type in self.move_funcs:
                    move = self.move_funcs[fallback_type](observation, archive_objs)
                else:
                    continue
                if move is not None and self._is_feasible(observation, move):
                    new_obs = observation.copy()
                    remove_idx, add_idx = move
                    new_obs[remove_idx] = 0
                    new_obs[add_idx] = 1
                    new_objs = np.sum(self.profits * new_obs, axis=1)
                    if archive_objs is not None and len(archive_objs) > 0:
                        improvement = not is_dominated(new_objs, archive_objs)
                    else:
                        improvement = False
                    if improvement:
                        self.pareto_improvements += 1
                    self.report_move_result(fallback_type, True, improvement)
                    return move
                else:
                    self.report_move_result(fallback_type, False, False)
            return self.mutation_move(observation, archive_objs)
    def act_batch(self, batch_obs, context=None):
        return [self.act(obs, context) for obs in batch_obs]
    def report_move_result(self, move_type, success, improvement=False):
        if move_type not in self.move_types:
            move_type = "mutation"
        self.move_counts[move_type] += 1
        if success and improvement:
            self.move_successes[move_type] += 1
        rates = np.array([self.move_successes[k] / self.move_counts[k] for k in self.move_types])
        exp_rates = np.exp(rates)
        new_probs = exp_rates / np.sum(exp_rates)
        if self.llm_generated_func is not None and self.move_successes["llm_generated"] > self.burn_in:
            self.move_probs = 0.7 * self.move_probs + 0.3 * new_probs
        else:
            self.move_probs = 0.8 * self.move_probs + 0.2 * new_probs
            self.move_probs[-1] = 0.01
        self.move_probs = self.move_probs / np.sum(self.move_probs)
    def report_batch_results(self, move_types, successes, improvements=None):
        if improvements is None:
            improvements = [False] * len(move_types)
        for mt, suc, imp in zip(move_types, successes, improvements):
            self.report_move_result(mt, suc, imp)
    def update_best(self, observation):
        self.best_observation = observation.copy()
    def get_stats(self, archive_objs=None, hv=None):
        diversity = float(np.mean(np.std(archive_objs, axis=0))) if archive_objs is not None and isinstance(archive_objs, np.ndarray) and archive_objs.ndim == 2 and archive_objs.shape[0] > 0 else 0.0
        move_success_rates = {k: float(self.move_successes[k]) / self.move_counts[k] for k in self.move_types}
        if hv is not None:
            self.hypervolume_history.append(hv)
            if len(self.hypervolume_history) > self.max_hv_length:
                self.hypervolume_history = self.hypervolume_history[-self.max_hv_length:]
        hv_delta = 0.0
        hv_mean = 0.0
        stagnation = False
        if len(self.hypervolume_history) > 1:
            hv_delta = self.hypervolume_history[-1] - self.hypervolume_history[-2]
            hv_mean = float(np.mean(self.hypervolume_history))
            if abs(hv_delta) < 1e-3:
                stagnation = True
        return {
            "generation": self.generation,
            "move_counts": dict(self.move_counts),
            "move_successes": dict(self.move_successes),
            "move_success_rates": move_success_rates,
            "move_probs": self.move_probs.tolist(),
            "archive_diversity": diversity,
            "hypervolume_current": self.hypervolume_history[-1] if self.hypervolume_history else None,
            "hypervolume_delta": hv_delta,
            "hypervolume_mean": hv_mean,
            "hv_stagnation": stagnation,
            "pareto_improvements": self.pareto_improvements,
            "last_feedback": self.last_feedback
        }
    # Moves (unchanged)
    def local_search_move(self, observation, archive_objs=None):
        items_selected = np.where(observation == 1)[0]
        items_unselected = np.where(observation == 0)[0]
        if len(items_selected) == 0 or len(items_unselected) == 0 or self.profits is None:
            return 0, 0
        profs_selected = np.sum(self.profits[:, items_selected], axis=0)
        profs_unselected = np.sum(self.profits[:, items_unselected], axis=0)
        remove_idx = items_selected[np.argmin(profs_selected)]
        add_idx = items_unselected[np.argmax(profs_unselected)]
        return int(remove_idx), int(add_idx)
    def mutation_move(self, observation, archive_objs=None):
        items_selected = np.where(observation == 1)[0]
        items_unselected = np.where(observation == 0)[0]
        if len(items_selected) == 0 or len(items_unselected) == 0:
            return 0, 0
        remove_idx = int(np.random.choice(items_selected))
        add_idx = int(np.random.choice(items_unselected))
        return remove_idx, add_idx
    def diversity_move(self, observation, archive_objs):
        best_move = None
        best_min_dist = -np.inf
        items_selected = np.where(observation == 1)[0]
        items_unselected = np.where(observation == 0)[0]
        if len(items_selected) == 0 or len(items_unselected) == 0 or archive_objs is None or archive_objs.shape[0] == 0:
            return None
        for _ in range(25):
            remove_idx = int(np.random.choice(items_selected))
            add_idx = int(np.random.choice(items_unselected))
            candidate = observation.copy()
            candidate[remove_idx] = 0
            candidate[add_idx] = 1
            candidate_objs = np.sum(self.profits * candidate, axis=1)
            dists = np.linalg.norm(archive_objs - candidate_objs, axis=1)
            min_dist = np.min(dists)
            if min_dist > best_min_dist:
                best_min_dist = min_dist
                best_move = (remove_idx, add_idx)
        return best_move
    def _is_feasible(self, observation, move):
        if self.weights is None or self.capacities is None or move is None:
            return False
        remove_idx, add_idx = move
        if remove_idx == add_idx:
            return False
        items = observation.copy()
        if items[remove_idx] == 0 or items[add_idx] == 1:
            return False
        items[remove_idx] = 0
        items[add_idx] = 1
        total_weights = np.sum(self.weights * items, axis=1)
        feasible = np.all(total_weights <= self.capacities)
        return feasible
    def gses_reflect(self, archive_objs, agent_stats, archive_items=None):
        print(f"Calling LLM reflect for generation {self.generation}")
        gaps = pareto_gaps(archive_objs)
        previous_code = self.llm_code_history[-1] if self.llm_code_history else None
        feedback = self.last_feedback if self.last_feedback else None
        code = prompt_llm_for_move_func(
            archive_objs, gaps, agent_stats, self.llm_endpoint,
            previous_code=previous_code, feedback=feedback
        )
        print("LLM raw response:", code)
        if code:
            if self.verbose: print("LLM-generated move code:\n", code)
            try:
                loc = {}
                import numpy as np
                exec(code, {"np": np, "is_dominated": is_dominated, "hypervolume": hypervolume, "pareto_gaps": pareto_gaps}, loc)
                llm_func = None
                for k in loc:
                    if callable(loc[k]):
                        llm_func = loc[k]
                        break
                if llm_func is not None:
                    score = 0
                    for _ in range(10):
                        if archive_items is not None and archive_items.shape[0] > 0:
                            idx = np.random.randint(archive_items.shape[0])
                            test_obs = archive_items[idx]
                        else:
                            test_obs = np.ones(self.num_items, dtype=np.int32)
                        try:
                            move = llm_func(test_obs, self.profits, self.weights, self.capacities, archive_objs)
                        except Exception as e:
                            print("LLM code error:", e)
                            continue
                        if move is not None and self._is_feasible(test_obs, move):
                            score += 1
                    if score > 7:
                        self.llm_generated_func = llm_func
                        self.move_types[-1] = "llm_generated"
                        self.move_funcs["llm_generated"] = self.llm_generated_func
                        self.move_probs[-1] = 0.1
                        self.move_probs = self.move_probs / np.sum(self.move_probs)
                        self.llm_code_history.append(code)
                        self.last_feedback = "LLM move integrated successfully (score: %d/10)" % score
                        print("LLM move integrated successfully.")
                    else:
                        self.last_feedback = "LLM move failed feasibility (score: %d/10)" % score
                        print("LLM-generated move is not feasible. Skipping integration.")
            except Exception as e:
                print("LLM exec error:", e)
        print(f"\n=== Diagnostics: Iteration {self.generation} ===")
        stats = self.get_stats(archive_objs)
        print("Move probabilities:", stats["move_probs"])
        print("Move success rates:", stats["move_success_rates"])
        print("Move counts:", stats["move_counts"])
        print("Hypervolume (current):", stats["hypervolume_current"])
        print("Hypervolume delta:", stats["hypervolume_delta"])
        print("Archive diversity:", stats["archive_diversity"])
        print("HV stagnation:", stats["hv_stagnation"])
        print("LLM move integrated:", self.llm_generated_func is not None)
        print("Feedback:", stats.get("last_feedback", ""))
        print("==============================")
    def maybe_reflect(self, archive_objs, archive_items=None):
        self.generation += 1
        agent_stats = self.get_stats(archive_objs)
        if archive_objs is not None and isinstance(archive_objs, np.ndarray) and archive_objs.ndim == 2 and archive_objs.shape[0] > 0:
            self.gses_reflect(archive_objs, agent_stats, archive_items=archive_items)

In [7]:
%%cython --annotate
from libc.stdlib cimport malloc, free, srand, rand
from libc.string cimport memset
from libc.math cimport exp

import numpy as np
cimport numpy as np

cdef public object hypervolume_func = None

def set_hypervolume_func(func):
    global hypervolume_func
    hypervolume_func = func

cdef struct ind:
    int nombr_nonpris
    int nombr
    int rank
    float fitnessbest
    float fitness
    int explored
    double *f
    double *capa
    double *v
    int *d
    int *Items

cdef struct pop:
    int size
    int maxsize
    ind **ind_array

cdef int NBITEMS = 250
cdef int ni = 250
cdef int L = 5        # FIXED to match code 1!
cdef double LARGE = 10e50
cdef float smallValue = 0.0000001
cdef double kappa = 0.05
cdef int alpha = 10   # FIXED to match code 1!
cdef int paretoIni = 28000  # FIXED to match code 1!

cdef int nf = 2
cdef double *capacities = NULL
cdef int **weights = NULL
cdef int **profits = NULL
cdef double *vector_weight = NULL
cdef double max_bound = 0.0
cdef double **OBJ_Weights = NULL
cdef int nombreLIGNE = 0
cdef int nextLn = 0
cdef int inv = 0
cdef int OBJ_Weights_lines = 0

cdef public object py_agent = None

def set_agent(agent):
    global py_agent
    py_agent = agent

def seed(int x):
    srand(x)

cdef int irand(int range_val):
    return rand() % range_val

cdef void *chk_malloc(size_t size):
    cdef void *return_value = malloc(size)
    if return_value == NULL:
        raise MemoryError("Out of memory.")
    memset(return_value, 0, size)
    return return_value

cdef pop *create_pop(int maxsize, int nf):
    cdef int i
    cdef pop *pp = <pop *>chk_malloc(sizeof(pop))
    pp.size = 0
    pp.maxsize = maxsize
    pp.ind_array = <ind **>chk_malloc(maxsize * sizeof(void*))
    for i in range(maxsize):
        pp.ind_array[i] = NULL
    return pp

cdef ind *create_ind(int nf):
    cdef int i
    cdef ind *p_ind = <ind *>chk_malloc(sizeof(ind))
    p_ind.nombr_nonpris = 0
    p_ind.nombr = 0
    p_ind.rank = 0
    p_ind.fitnessbest = -1.0
    p_ind.fitness = -1.0
    p_ind.explored = 0
    p_ind.f = <double *>chk_malloc(nf * sizeof(double))
    p_ind.capa = <double *>chk_malloc(nf * sizeof(double))
    p_ind.v = <double *>chk_malloc(nf * sizeof(double))
    p_ind.d = <int *>chk_malloc(ni * sizeof(int))
    p_ind.Items = <int *>chk_malloc(ni * sizeof(int))
    for i in range(ni):
        p_ind.Items[i] = 0
        p_ind.d[i] = 0
    for i in range(nf):
        p_ind.f[i] = 0.0
        p_ind.capa[i] = 0.0
        p_ind.v[i] = 0.0
    return p_ind

cdef ind *ind_copy(ind *i):
    cdef ind *p_ind = create_ind(nf)
    cdef int k
    p_ind.nombr_nonpris = i.nombr_nonpris
    p_ind.nombr = i.nombr
    p_ind.rank = i.rank
    p_ind.fitnessbest = i.fitnessbest
    p_ind.fitness = i.fitness
    p_ind.explored = i.explored
    for k in range(nf):
        p_ind.f[k] = i.f[k]
        p_ind.v[k] = i.v[k]
        p_ind.capa[k] = i.capa[k]
    for k in range(ni):
        p_ind.d[k] = i.d[k]
        p_ind.Items[k] = i.Items[k]
    return p_ind

cdef void free_ind(ind *p_ind):
    if p_ind != NULL:
        free(p_ind.d)
        free(p_ind.f)
        free(p_ind.capa)
        free(p_ind.v)
        free(p_ind.Items)
        free(p_ind)

cdef void complete_free_pop(pop *pp):
    cdef int i
    if pp != NULL:
        if pp.ind_array != NULL:
            for i in range(pp.size):
                if pp.ind_array[i] != NULL:
                    free_ind(pp.ind_array[i])
                    pp.ind_array[i] = NULL
            free(pp.ind_array)
        free(pp)

def cleanup_globals():
    global capacities, weights, profits, vector_weight, OBJ_Weights, OBJ_Weights_lines, nf, ni
    if capacities != NULL:
        free(capacities)
        capacities = NULL
    if weights != NULL:
        for i in range(nf):
            if weights[i] != NULL:
                free(weights[i])
        free(weights)
        weights = NULL
    if profits != NULL:
        for i in range(nf):
            if profits[i] != NULL:
                free(profits[i])
        free(profits)
        profits = NULL
    if vector_weight != NULL:
        free(vector_weight)
        vector_weight = NULL
    if OBJ_Weights != NULL:
        for i in range(nf):
            if OBJ_Weights[i] != NULL:
                free(OBJ_Weights[i])
        free(OBJ_Weights)
        OBJ_Weights = NULL
    OBJ_Weights_lines = 0
    nf = 0
    ni = 0

def loadMOKP(filename):
    global nf, ni, capacities, weights, profits
    cdef int i, f
    with open(filename.decode(), "r") as source:
        _nf, _ni = [int(x) for x in source.readline().split()]
        nf = _nf
        ni = _ni
        capacities = <double *>chk_malloc(nf * sizeof(double))
        weights = <int **>chk_malloc(nf * sizeof(void*))
        profits = <int **>chk_malloc(nf * sizeof(void*))
        for f in range(nf):
            capacities[f] = float(source.readline().strip())
            weights[f] = <int *>chk_malloc(ni * sizeof(int))
            profits[f] = <int *>chk_malloc(ni * sizeof(int))
            for i in range(ni):
                source.readline()
                weights[f][i] = int(source.readline().strip())
                profits[f][i] = int(source.readline().strip())

def read_weights_file(filename):
    global OBJ_Weights, nombreLIGNE, nf, OBJ_Weights_lines
    cdef int i, j, nlines
    with open(filename.decode(), "r") as f:
        lines = [line for line in f if line.strip()]
    nlines = len(lines)
    OBJ_Weights = <double **>chk_malloc(nf * sizeof(void*))
    for i in range(nf):
        OBJ_Weights[i] = <double *>chk_malloc(nlines * sizeof(double))
    for i, line in enumerate(lines):
        vals = line.strip().split()
        for j in range(nf):
            OBJ_Weights[j][i] = float(vals[j])
    nombreLIGNE = nlines - 1
    OBJ_Weights_lines = nlines

cdef void dynamic_weight_allpop():
    global vector_weight, OBJ_Weights, nombreLIGNE, nf, nextLn
    cdef int i
    if vector_weight == NULL:
        vector_weight = <double *>chk_malloc(nf * sizeof(double))
    for i in range(nf):
        vector_weight[i] = OBJ_Weights[i][nextLn]
    if nextLn == nombreLIGNE:
        nextLn = 0
    else:
        nextLn += 1

def choose_weight():
    dynamic_weight_allpop()

cdef void random_init_ind(ind *x):
    cdef int j, r, tmp
    for j in range(ni):
        x.d[j] = j
    for j in range(ni):
        r = irand(ni)
        tmp = x.d[r]
        x.d[r] = x.d[j]
        x.d[j] = tmp

cdef void evaluate(ind *x):
    cdef int j, l, k, faisable
    x.nombr = 0
    x.nombr_nonpris = 0
    for j in range(nf):
        x.capa[j] = 0.0
        x.f[j] = 0.0
    for j in range(ni):
        l = 0
        faisable = 1
        while l < nf and faisable == 1:
            if x.capa[l] + weights[l][x.d[j]] > capacities[l]:
                faisable = 0
            l += 1
        if faisable == 1:
            for k in range(nf):
                x.capa[k] += weights[k][x.d[j]]
                x.f[k] += profits[k][x.d[j]]
            x.Items[x.d[j]] = 1
            x.nombr += 1
        else:
            x.Items[x.d[j]] = 0
            x.nombr_nonpris += 1

cdef void P_init_pop(pop *SP, pop *Sarchive, int alpha):
    cdef int i, x, tmp, t
    t = max(alpha, Sarchive.size)
    cdef int* shuffle = <int *>chk_malloc(t * sizeof(int))
    for i in range(t):
        shuffle[i] = i
    for i in range(t):
        x = irand(alpha)
        tmp = shuffle[i]
        shuffle[i] = shuffle[x]
        shuffle[x] = tmp
    SP.size = alpha
    if Sarchive.size > alpha:
        for i in range(alpha):
            SP.ind_array[i] = ind_copy(Sarchive.ind_array[shuffle[i]])
    else:
        for i in range(alpha):
            if shuffle[i] < Sarchive.size:
                SP.ind_array[i] = ind_copy(Sarchive.ind_array[shuffle[i]])
            else:
                SP.ind_array[i] = create_ind(nf)
                random_init_ind(SP.ind_array[i])
                evaluate(SP.ind_array[i])
    free(shuffle)

cdef int non_dominated(ind *p_ind_a, ind *p_ind_b):
    cdef int i
    cdef int a_is_good = -1
    cdef int equal = 1
    for i in range(nf):
        if p_ind_a.f[i] > p_ind_b.f[i]:
            a_is_good = 1
        if p_ind_a.f[i] != p_ind_b.f[i]:
            equal = 0
    if equal:
        return 0
    return a_is_good

cdef double calcAddEpsIndicator(ind *p_ind_a, ind *p_ind_b):
    global max_bound
    cdef int i
    cdef double eps
    cdef double temp_eps
    if max_bound == 0.0:
        max_bound = 1e-8
    eps = (p_ind_a.v[0]/max_bound)-(p_ind_b.v[0]/max_bound)
    for i in range(1, nf):
        temp_eps = (p_ind_a.v[i]/max_bound)-(p_ind_b.v[i]/max_bound)
        if temp_eps > eps:
            eps = temp_eps
    return eps

cdef void init_fitness(ind *x):
    x.fitness = 0.0

cdef void update_fitness(ind *x, double I):
    x.fitness -= exp(-I / kappa)

cdef double update_fitness_return(double f, double I):
    return f - exp(-I / kappa)

cdef int delete_fitness(ind *x, double I):
    x.fitness += exp(-I / kappa)
    return 0

cdef void compute_ind_fitness(ind *x, pop *SP):
    cdef int j
    init_fitness(x)
    for j in range(SP.size):
        if SP.ind_array[j] != x:
            update_fitness(x, calcAddEpsIndicator(SP.ind_array[j], x))

cdef void compute_all_fitness(pop *SP):
    cdef int i
    for i in range(SP.size):
        compute_ind_fitness(SP.ind_array[i], SP)

cdef int extractPtoArchive(pop *P, pop *archive):
    cdef int i, j, dom, t, convergence_rate
    t = archive.size + P.size
    archiveAndP = create_pop(t, nf)
    convergence_rate = 0
    for i in range(archive.size):
        archiveAndP.ind_array[i] = archive.ind_array[i]
    for i in range(P.size):
        archiveAndP.ind_array[i + archive.size] = ind_copy(P.ind_array[i])
    archiveAndP.size = t
    archive.size = 0
    for i in range(t):
        for j in range(t):
            if i != j:
                dom = non_dominated(archiveAndP.ind_array[i], archiveAndP.ind_array[j])
                if dom == -1 or (dom == 0 and i > j):
                    break
        else:
            archive.ind_array[archive.size] = ind_copy(archiveAndP.ind_array[i])
            archive.size += 1
            if i >= t - P.size:
                convergence_rate += 1
    complete_free_pop(archiveAndP)
    return convergence_rate

cdef double calcMaxbound(pop *SP, int size):
    global max_bound
    cdef int i, j
    SP.size = size
    cdef double max_b = SP.ind_array[0].v[0]
    for i in range(SP.size):
        for j in range(nf):
            if max_b < SP.ind_array[i].v[j]:
                max_b = SP.ind_array[i].v[j]
    if max_b == 0.0:
        max_b = 1e-8
    max_bound = max_b
    return max_b

cdef void calcul_weight(pop *SP, int size):
    cdef int i, j
    for i in range(SP.size):
        for j in range(nf):
            SP.ind_array[i].v[j] = SP.ind_array[i].f[j] * vector_weight[j]

cdef int compute_fitness_and_select(pop *SP, ind *x, int size):
    cdef int i, worst
    cdef double worst_fit, fit_tmp
    SP.size = size
    x.fitness = 0
    compute_ind_fitness(x, SP)
    worst_fit = x.fitness
    worst = -1
    for i in range(SP.size):
        fit_tmp = update_fitness_return(SP.ind_array[i].fitness, calcAddEpsIndicator(x, SP.ind_array[i]))
        if fit_tmp > worst_fit:
            worst = i
            worst_fit = fit_tmp
    fit_tmp = x.fitness
    if worst == -1:
        return -1
    else:
        for i in range(SP.size):
            delete_fitness(SP.ind_array[i], calcAddEpsIndicator(SP.ind_array[worst], SP.ind_array[i]))
            update_fitness(SP.ind_array[i], calcAddEpsIndicator(x, SP.ind_array[i]))
        delete_fitness(x, calcAddEpsIndicator(SP.ind_array[worst], x))
        free_ind(SP.ind_array[worst])
        SP.ind_array[worst] = ind_copy(x)
        if fit_tmp - worst_fit > smallValue:
            return worst
        else:
            return -1

cdef np.ndarray get_archive_objs(pop *archive):
    cdef int i, j
    arr = np.zeros((archive.size, nf), dtype=np.float64)
    for i in range(archive.size):
        for j in range(nf):
            arr[i, j] = archive.ind_array[i].f[j]
    return arr

cdef np.ndarray get_items(ind *x):
    arr = np.zeros(ni, dtype=np.int32)
    for i in range(ni):
        arr[i] = x.Items[i]
    return arr

cdef void Indicator_local_search1(pop *SP, pop *Sarchive, int size):
    cdef ind *x
    cdef ind *y
    cdef int i, j, r, t, k, l, v, sol, mino, mp, maxp, consistant, pos, stop, convergence, ii, tmp_pris, tmp_nonpris, taille, feasible, tv, IM
    cdef int* remplace = <int *>chk_malloc(L * sizeof(int))
    SP.size = size
    extractPtoArchive(SP, Sarchive)
    while True:
        convergence = 0
        archive_objs = get_archive_objs(Sarchive) if Sarchive.size > 0 else None
        for i in range(SP.size):
            if not SP.ind_array[i].explored:
                x = ind_copy(SP.ind_array[i])
                j = 0
                while j < x.nombr:
                    for l in range(L):
                        remplace[l] = 0
                    # -------- CLASSIC AGENT INTEGRATION --------
                    remove_idx = -1
                    add_idx = -1
                    agent_move_type = None
                    agent_success = False
                    if py_agent is not None:
                        state = {
                            "Items": [x.Items[ii] for ii in range(ni)],
                            "capa": [x.capa[ii] for ii in range(nf)],
                            "f": [x.f[ii] for ii in range(nf)],
                        }
                        context = {}
                        if archive_objs is not None:
                            context["archive_objs"] = archive_objs
                        try:
                            obs = py_agent.observe(state)
                            remove_idx, add_idx = py_agent.act(obs, context)
                        except Exception as e:
                            print("Agent error:", e)
                            remove_idx = -1
                            add_idx = -1
                    # Fallback to random if agent does not provide usable indices
                    if remove_idx < 0 or remove_idx >= ni or x.Items[remove_idx] != 1:
                        while True:
                            mino = irand(ni)
                            if x.Items[mino] == 1:
                                break
                        remove_idx = mino
                    if add_idx < 0 or add_idx >= ni or x.Items[add_idx] != 0 or add_idx == remove_idx:
                        while True:
                            maxp = irand(ni)
                            if x.Items[maxp] == 0 and maxp != remove_idx:
                                break
                        add_idx = maxp

                    # Remove item
                    x.Items[remove_idx] = 0
                    x.nombr -= 1
                    x.nombr_nonpris += 1
                    for r in range(nf):
                        x.capa[r] -= weights[r][remove_idx]
                        x.f[r] -= profits[r][remove_idx]

                    IM = 0
                    taille = 0
                    while IM < L:
                        item_to_add = add_idx
                        if item_to_add < 0 or item_to_add >= ni or x.Items[item_to_add] != 0 or item_to_add == remove_idx:
                            while True:
                                item_to_add = irand(ni)
                                if x.Items[item_to_add] == 0 and item_to_add != remove_idx:
                                    break
                        consistant = 1
                        r = 0
                        while r < nf and consistant == 1:
                            if x.capa[r] + weights[r][item_to_add] > capacities[r]:
                                consistant = 0
                            r += 1
                        if consistant == 1:
                            feasible = 1
                            r = 0
                            while r < taille and feasible:
                                if item_to_add == remplace[r]:
                                    feasible = 0
                                r += 1
                            if feasible == 1:
                                remplace[taille] = item_to_add
                                taille += 1
                                x.Items[item_to_add] = 1
                                x.nombr_nonpris -= 1
                                x.nombr += 1
                                for r in range(nf):
                                    x.capa[r] += weights[r][item_to_add]
                                    x.f[r] += profits[r][item_to_add]
                        IM += 1
                    for tv in range(nf):
                        x.v[tv] = x.f[tv] * vector_weight[tv]
                    max_bound = calcMaxbound(SP, SP.size)
                    sol = compute_fitness_and_select(SP, x, SP.size)
                    agent_success = (sol != -1)
                    if py_agent is not None:
                        try:
                            py_agent.report_move_result(None, agent_success)
                        except Exception:
                            pass
                    if sol != -1:
                        j = x.nombr + 1
                        if sol > i and i + 1 < SP.size:
                            y = SP.ind_array[i + 1]
                            SP.ind_array[i + 1] = SP.ind_array[sol]
                            SP.ind_array[sol] = y
                            i += 1
                        break
                    elif sol == -1:
                        x.Items[remove_idx] = 1
                        x.nombr_nonpris -= 1
                        x.nombr += 1
                        for r in range(nf):
                            x.capa[r] += weights[r][remove_idx]
                            x.f[r] += profits[r][remove_idx]
                        if taille >= 1:
                            for r in range(taille):
                                x.Items[remplace[r]] = 0
                                x.nombr -= 1
                                x.nombr_nonpris += 1
                                for t in range(nf):
                                    x.capa[t] -= weights[t][remplace[r]]
                                    x.f[t] -= profits[t][remplace[r]]
                                    x.v[t] = x.f[t] * vector_weight[t]
                    j += 1
                tmp_pris = x.nombr
                tmp_nonpris = x.nombr_nonpris
                free_ind(x)
                if j == tmp_pris:
                    SP.ind_array[i].explored = 1
        convergence = extractPtoArchive(SP, Sarchive)
        if not convergence:
            break
    free(remplace)

def run_moacp(instance_file, weights_file, nbitems, num_objectives, output_file):
    global nf, ni, NBITEMS, alpha, paretoIni, L, nombreLIGNE, nextLn, inv, vector_weight
    global capacities, weights, profits, OBJ_Weights

    alpha = 10
    paretoIni = 28000

    NBL = 100
    NRUNS = 10

    for run in range(1, NRUNS+1):
        NBITEMS = nbitems
        ni = nbitems
        nf = num_objectives

        print(f"RUN {run}/{NRUNS} -- {instance_file.decode()} nbitems={ni} nf={nf} => {output_file}")

        nombreLIGNE = 0
        nextLn = 0
        inv = 0

        seed(run)
        loadMOKP(instance_file)
        read_weights_file(weights_file)

        vector_weight = <double *>chk_malloc(nf * sizeof(double))
        P = create_pop(paretoIni, nf)

        it = 0
        while it < NBL:
            solutions = create_pop(alpha, nf)
            archive = create_pop(paretoIni, nf)
            choose_weight()
            P_init_pop(solutions, P, alpha)
            extractPtoArchive(solutions, P)
            calcul_weight(solutions, alpha)
            calcMaxbound(solutions, alpha)
            compute_all_fitness(solutions)
            Indicator_local_search1(solutions, archive, alpha)
            extractPtoArchive(archive, P)
            # Update agent with best observation (MHRE-style best-following)
            if py_agent is not None and P.size > 0:
                best_idx = 0
                best_val = P.ind_array[0].fitness
                for i in range(1, P.size):
                    if P.ind_array[i].fitness > best_val:
                        best_idx = i
                        best_val = P.ind_array[i].fitness
                best_items = get_items(P.ind_array[best_idx])
                try:
                    py_agent.update_best(best_items)
                except Exception:
                    pass
            it += 1
            complete_free_pop(solutions)
            complete_free_pop(archive)

        with open(output_file, "a") as fpareto:
            fpareto.write("\n")
            for i in range(P.size):
                for j in range(nf):
                    fpareto.write(f"{P.ind_array[i].f[j]:.6f} ")
                fpareto.write("\n")

        complete_free_pop(P)
        cleanup_globals()

In [8]:
nbitems = 250
num_objectives = 2
instance_file = "250.2.txt"
weights_file = "Weights_2obj_FQ200.txt"
output_file_agent = "oohh.txt"

profits = load_profits_from_file(instance_file, num_objectives, nbitems)
weights = load_weights_from_file(instance_file, num_objectives, nbitems)
capacities = load_capacities_from_file(instance_file, num_objectives, nbitems)

agent = MHREGSESLlmAgent(
    nbitems, num_objectives,
    profits=profits,
    weights=weights,
    capacities=capacities,
    meta_interval=1,    # reflect every iteration for demo!
    llm_endpoint="http://localhost:11434/api/generate", # or your LLM endpoint
    burn_in=0,
    verbose=True
)

set_agent(agent)
set_hypervolume_func(calculate_hypervolume)
run_moacp(instance_file.encode(), weights_file.encode(), nbitems, num_objectives, output_file_agent)
print("Optimization completed.")

RUN 1/10 -- 250.2.txt nbitems=250 nf=2 => oohh.txt


KeyboardInterrupt: 